In [41]:
import optuna
import torch.nn as nn

from models import (
    ResNet18,
    ResNet34,
    ResNet50,
    PreActResNet18,
    PreActResNet34,
    PreActResNet50,
    DenseNet121,
    DenseNet161,
    DenseNet169,
)
from utils import (
    get_cifar10_train_val_loaders,
    get_device,
    get_optimiser,
    get_scheduler,
    pickle_dump,
    train, 
    test,
)

In [30]:
hp_floats = {
    "lr": {"low": 1e-4, "high": 0.1, "log": True},
    "momentum": {"low": 0.4, "high": 0.9},
    "weight_decay": {"low": 5e-4, "high": 1e-3},
}

hp_categorical = {
    "optimisers": {"names": ["SGD", "Adam", "RMSprop", "AdamW"]},
    "lr_schedulers": {
        "names": ["StepLR", "ExponentialLR", "ReduceLROnPlateau", "CosineAnnealingLR"],
        "params": {
            "StepLR": {"step_size": 5, "gamma": 0.5},
            "ExponentialLR": {"gamma": 0.9},
            "ReduceLROnPlateau": {"mode": 'min', "factor": 0.5, "patience": 3},
            "CosineAnnealingLR": {"T_max": 10},
        }
    },
}


In [39]:
def objective(trial, model_class: nn.Module, n_epochs: int = 10, device: str = "cuda") -> float:
    algo_hyperparams = {
        hp_name: trial.suggest_float(hp_name, **hp_params)
        for hp_name, hp_params in hp_floats.items()
    }

    optimiser_name = trial.suggest_categorical("optimizer", hp_categorical["optimisers"]["names"])
    scheduler_name = trial.suggest_categorical("lr_scheduler", hp_categorical["lr_schedulers"]["names"])

    train_loader, val_loader = get_cifar10_train_val_loaders()

    model = model_class().to(device)

    if optimiser_name in ("Adam", "AdamW"):
        del algo_hyperparams["momentum"]

    optimiser = get_optimiser(
        optimiser_name, model.parameters(), **algo_hyperparams,
    )
    scheduler = get_scheduler(
        scheduler_name,
        optimiser,
        **hp_categorical["lr_schedulers"]["params"][scheduler_name],
    )
    criterion = nn.CrossEntropyLoss()

    best_val_acc = 0
    for epoch in range(n_epochs):
        train_acc, train_loss = train(train_loader, model, optimiser, criterion, device)
        val_acc, val_loss = test(val_loader, model, criterion, device)

        if scheduler_name == "ReduceLROnPlateau":
            scheduler.step(val_loss)
        else:
            scheduler.step()

        trial.report(val_acc, epoch)

        if val_acc > best_val_acc:
            best_val_acc = val_acc

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return best_val_acc


[I 2025-04-21 17:06:18,343] A new study created in memory with name: no-name-e74983e7-4146-400c-9a19-9df7e19a9c93
[I 2025-04-21 17:07:36,015] Trial 0 finished with value: 12.6 and parameters: {'lr': 0.01670994484273603, 'momentum': 0.8554835164978709, 'weight_decay': 0.0007308598285075895, 'optimizer': 'RMSprop', 'lr_scheduler': 'StepLR'}. Best is trial 0 with value: 12.6.
[I 2025-04-21 17:08:52,905] Trial 1 finished with value: 67.22 and parameters: {'lr': 0.00016241757677590437, 'momentum': 0.656749040191158, 'weight_decay': 0.000989965326603325, 'optimizer': 'Adam', 'lr_scheduler': 'CosineAnnealingLR'}. Best is trial 1 with value: 67.22.
[I 2025-04-21 17:10:09,289] Trial 2 finished with value: 18.58 and parameters: {'lr': 0.08934223247950826, 'momentum': 0.8473421104744163, 'weight_decay': 0.0006923527135673363, 'optimizer': 'AdamW', 'lr_scheduler': 'StepLR'}. Best is trial 1 with value: 67.22.
[I 2025-04-21 17:11:22,401] Trial 3 finished with value: 62.26 and parameters: {'lr': 0.0

KeyboardInterrupt: 

In [36]:
import torch.optim